In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

from sklearn.metrics import r2_score

In [7]:
df = pd.read_csv('master_df.csv')
df.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,zipcode,lat,long,sqft_living15,sqft_lot15,basement,Renovated,year,month,age_when_sold
0,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,3,...,98125,47.7210,-122.319,1690,7639,1,1,2014,12,63
1,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,3,...,98028,47.7379,-122.233,2720,8062,0,0,2015,2,82
2,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,5,...,98136,47.5208,-122.393,1360,5000,1,0,2014,12,49
3,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,3,...,98074,47.6168,-122.045,1800,7503,0,0,2015,2,28
4,2014-05-12,1230000.0,4,4.50,5420,101930,1.0,0,0,3,...,98053,47.6561,-122.005,4760,101930,1,0,2014,5,13


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19054 entries, 0 to 19053
Data columns (total 24 columns):
date             19054 non-null object
price            19054 non-null float64
bedrooms         19054 non-null int64
bathrooms        19054 non-null float64
sqft_living      19054 non-null int64
sqft_lot         19054 non-null int64
floors           19054 non-null float64
view             19054 non-null int64
condition        19054 non-null int64
grade            19054 non-null int64
sqft_above       19054 non-null int64
sqft_basement    19054 non-null float64
yr_built         19054 non-null int64
yr_renovated     19054 non-null int64
zipcode          19054 non-null int64
lat              19054 non-null float64
long             19054 non-null float64
sqft_living15    19054 non-null int64
sqft_lot15       19054 non-null int64
basement         19054 non-null int64
Renovated        19054 non-null int64
year             19054 non-null int64
month            19054 non-null int64
age_

In [8]:
# df.drop('date', axis = 1, inplace = True)
df.drop('waterfront', axis = 1, inplace = True)

In [9]:
sk_ols = LinearRegression(fit_intercept=False, normalize=False, n_jobs=-1)

In [10]:
rfe = RFE(sk_ols, n_features_to_select=10, step=1, verbose=2)

In [31]:
X_all = df.drop(columns=['price', 'month', 'yr_renovated', 'yr_built'])
rfe.fit(X_all, df['price'])

Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.


RFE(estimator=LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1,
                               normalize=True),
    n_features_to_select=10, step=1, verbose=2)

In [28]:
X_all.head()

,bedrooms,bathrooms,sqft_living,sqft_lot,floors,view,condition,grade,sqft_above,sqft_basement,zipcode,lat,long,sqft_living15,sqft_lot15,basement,Renovated,year,age_when_sold
0,3,2.25,2570,7242,2.0,0,3,7,2170,400.0,98125,47.7210,-122.319,1690,7639,1,1,2014,63
1,2,1.00,770,10000,1.0,0,3,6,770,0.0,98028,47.7379,-122.233,2720,8062,0,0,2015,82
2,4,3.00,1960,5000,1.0,0,5,7,1050,910.0,98136,47.5208,-122.393,1360,5000,1,0,2014,49
3,3,2.00,1680,8080,1.0,0,3,8,1680,0.0,98074,47.6168,-122.045,1800,7503,0,0,2015,28
4,4,4.50,5420,101930,1.0,0,3,11,3890,1530.0,98053,47.6561,-122.005,4760,101930,1,0,2014,13


In [32]:
new_cols = X_all.columns[rfe.support_]

In [8]:
X_new = df[new_cols]
X_new.head()

,bedrooms,bathrooms,view,condition,grade,lat,long,basement,Renovated,year
0,3,2.25,0,3,7,47.7210,-122.319,1,1,2014
1,2,1.00,0,3,6,47.7379,-122.233,0,0,2015
2,4,3.00,0,5,7,47.5208,-122.393,1,0,2014
3,3,2.00,0,3,8,47.6168,-122.045,0,0,2015
4,4,4.50,0,3,11,47.6561,-122.005,1,0,2014


In [9]:
def make_ols_sklearn(X, y, test_size=0.20, fit_intercept=False, standardize=False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    if standardize:
        ss = StandardScaler()
        ss.fit(X_train)
        X_train = ss.transform(X_train)
        X_test = ss.transform(X_test)
    ols = LinearRegression(fit_intercept=fit_intercept, normalize=False)
    ols.fit(X_train, y_train)
    train_score = ols.score(X_train, y_train)
    test_score = ols.score(X_test, y_test)
    print(f"train score = {train_score}")
    print(f"test score = {test_score}")
    return ols

In [10]:
make_ols_sklearn(X_new, df['price'])

train score = 0.605941611518015
test score = 0.6163737981343089


LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [11]:
cv_scores = cross_val_score(sk_ols, X_new, df['price'], scoring='r2', cv=10, n_jobs=-1) 

In [12]:
cv_scores

array([0.61408544, 0.60120655, 0.59632493, 0.58451427, 0.59889095,
       0.62840211, 0.60892371, 0.62812159, 0.60834726, 0.57946449])

In [13]:
target ='price'

In [14]:
train, test = train_test_split(df)

In [15]:
predictors = '+'.join(new_cols)
formula = target + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.608
Method:                 Least Squares   F-statistic:                     2216.
Date:                Tue, 02 Jun 2020   Prob (F-statistic):               0.00
Time:                        19:18:25   Log-Likelihood:            -1.9499e+05
No. Observations:               14290   AIC:                         3.900e+05
Df Residuals:                   14279   BIC:                         3.901e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.013e+08   7.57e+06    -13.387      0.000   -1.16e+08   -8.65e+07
bedrooms    2.118e+04   2264.949      9.353      0.000    1.67e+04    2.56e+04
bathrooms   5.348e+04   3375.913     15.842      0.000    4.69e+04    6.01e+04
view        7.688e+04   2592.059     29.659      0.000    7.18e+04     8.2e+04
condition   6.546e+04   2715.425     24.109      0.000    6.01e+04    7.08e+04
grade        1.51e+05   2050.465     73.626      0.000    1.47e+05    1.55e+05
lat         6.149e+05   1.27e+04     48.549      0.000     5.9e+05     6.4e+05
long       -9.924e+04   1.32e+04     -7.519      0.000   -1.25e+05   -7.34e+04
basement    1.987e+04   3805.884      5.222      0.000    1.24e+04    2.73e+04
Renovated   1.416e+05   9561.284     14.814      0.000    1.23e+05     1.6e+05
year        2.923e+04   3658.784      7.990      0.000    2.21e+04    3.64e+04
==============================================================================
Omnibus:                     8778.977   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           158654.977
Skew:                           2.633   Prob(JB):                         0.00
Kurtosis:                      18.451   Cond. No.                     8.95e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.95e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [16]:
X = X_new[new_cols]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(new_cols, vif))

[('bedrooms', 21.387635472313328),
 ('bathrooms', 19.544531619313823),
 ('view', 1.197403082134349),
 ('condition', 30.477814849157824),
 ('grade', 85.83033438764114),
 ('lat', 123899.38533889216),
 ('long', 855634.7170872815),
 ('basement', 1.905413518127788),
 ('Renovated', 1.0488612194733118),
 ('year', 885576.1030736113)]

In [17]:
x_test2 = pd.DataFrame()

In [25]:
def norm_feat(series):
    return (series - series.mean())/series.std()
for feat in X_all:
    x_test2[feat] = norm_feat(df[feat])
x_test2.describe()

,bedrooms,bathrooms,view,condition,grade,lat,long,basement,Renovated,year,sqft_living,sqft_lot,floors,waterfront,sqft_above,sqft_basement,zipcode,sqft_living15,sqft_lot15,age_when_sold
count,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,0.0,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04,1.905400e+04
mean,-2.655119e-16,2.625286e-16,2.386624e-17,-3.042946e-16,2.088296e-16,-3.835976e-14,8.879882e-14,2.834116e-17,2.535788e-17,-1.402615e-13,2.386624e-16,-1.044148e-17,-2.386624e-17,NaN,2.983280e-17,-4.176592e-17,-5.021457e-14,1.163479e-16,1.789968e-17,-4.773248e-17
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.632341e+00,-2.123568e+00,-2.907715e-01,-3.709074e+00,-4.023604e+00,-2.909253e+00,-2.169913e+00,-7.874438e-01,-1.835424e-01,-6.896786e-01,-1.902042e+00,-3.544577e-01,-9.140797e-01,NaN,-1.734702e+00,-6.491640e-01,-1.435703e+00,-2.333260e+00,-4.372463e-01,-1.504963e+00
25%,-4.126776e-01,-4.744082e-01,-2.907715e-01,-6.299220e-01,-5.600898e-01,-6.416476e-01,-8.132306e-01,-7.874438e-01,-1.835424e-01,-6.896786e-01,-7.260457e-01,-2.439011e-01,-9.140797e-01,NaN,-7.270813e-01,-6.491640e-01,-8.547154e-01,-7.232049e-01,-2.760580e-01,-8.839964e-01
50%,-4.126776e-01,1.852557e-01,-2.907715e-01,-6.299220e-01,-5.600898e-01,8.638338e-02,-1.171321e-01,-7.874438e-01,-1.835424e-01,-6.896786e-01,-1.772473e-01,-1.813151e-01,1.200498e-02,NaN,-2.724233e-01,-6.491640e-01,-2.362444e-01,-2.066886e-01,-1.854824e-01,-1.439400e-01
75%,6.971543e-01,5.150876e-01,-2.907715e-01,9.096539e-01,3.057886e-01,8.509509e-01,6.357909e-01,1.269865e+00,-1.835424e-01,1.449875e+00,5.283507e-01,-1.079927e-01,9.380897e-01,NaN,5.140121e-01,5.780819e-01,7.383160e-01,5.459494e-01,-9.774178e-02,6.726740e-01
max,8.465977e+00,7.771390e+00,5.468439e+00,2.449230e+00,3.769302e+00,1.566563e+00,6.382156e+00,1.269865e+00,5.448047e+00,1.449875e+00,1.284832e+01,4.011139e+01,3.716344e+00,NaN,9.373699e+00,8.914092e+00,2.275123e+00,6.242386e+00,3.110295e+01,2.442004e+00


In [151]:
def log_turn(series):
    for x in series:
        return np.log(x)

In [44]:
sk_ols = LinearRegression(fit_intercept=False, normalize=False, n_jobs=-1)

In [157]:
rfe = RFE(sk_ols, n_features_to_select=10, step=1, verbose=2)
# df['lat_log'] = norm_feat(df['lat'])
# df['long_log'] = norm_feat(df['long'])
df['sqft_living_log'] = norm_feat(df['sqft_living'])

In [158]:
X_all = df.drop(columns=['price', 'lat', 'long', 'month', 'sqft_living'])
rfe.fit(X_all, df['price'])

Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.


RFE(estimator=LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1,
                               normalize=False),
    n_features_to_select=10, step=1, verbose=2)

In [142]:
new_cols = X_all.columns[rfe.support_]

In [143]:
X_new = df[new_cols]
X_new.head()

,bedrooms,bathrooms,view,condition,grade,basement,Renovated,lat_log,long_log,sqft_living_log
0,3,2.25,0,3,7,1,1,1.159082,-0.749303,0.561951
1,2,1.00,0,3,6,0,0,1.280750,-0.138441,-1.454044
2,4,3.00,0,5,7,1,0,-0.282222,-1.274929,-0.121247
3,3,2.00,0,3,8,0,0,0.408913,1.196932,-0.434847
4,4,4.50,0,3,11,1,0,0.691846,1.481053,3.753942


In [144]:
make_ols_sklearn(X_new, df['price'])

train score = 0.6636901800918394
test score = 0.6519056259664313


LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [145]:
cv_scores = cross_val_score(sk_ols, X_new, df['price'], scoring='r2', cv=10, n_jobs=-1) 
cv_scores

array([0.65686744, 0.65172209, 0.6417641 , 0.63667299, 0.64861542,
       0.66918308, 0.65742959, 0.6680759 , 0.67684009, 0.67129357])

In [146]:
train, test = train_test_split(df)

In [147]:
predictors = '+'.join(new_cols)
formula = target + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.665
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     2832.
Date:                Wed, 03 Jun 2020   Prob (F-statistic):               0.00
Time:                        09:32:09   Log-Likelihood:            -1.9397e+05
No. Observations:               14290   AIC:                         3.880e+05
Df Residuals:                   14279   BIC:                         3.881e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -2.9e+05   2.23e+04    -12.984      0.000   -3.34e+05   -2.46e+05
bedrooms        -2.489e+04   2295.030    -10.843      0.000   -2.94e+04   -2.04e+04
bathrooms       -1850.3497   3354.214     -0.552      0.581   -8425.047    4724.347
view             5.726e+04   2423.631     23.628      0.000    5.25e+04     6.2e+04
condition        5.725e+04   2533.674     22.596      0.000    5.23e+04    6.22e+04
grade            9.149e+04   2271.311     40.279      0.000     8.7e+04    9.59e+04
basement        -8350.6182   3604.624     -2.317      0.021   -1.54e+04   -1285.085
Renovated        1.188e+05   8985.143     13.219      0.000    1.01e+05    1.36e+05
lat_log          8.555e+04   1638.494     52.214      0.000    8.23e+04    8.88e+04
long_log        -2.923e+04   1765.334    -16.560      0.000   -3.27e+04   -2.58e+04
sqft_living_log   1.57e+05   3224.694     48.686      0.000    1.51e+05    1.63e+05
==============================================================================
Omnibus:                     8135.146   Durbin-Watson:                   2.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           139755.232
Skew:                           2.382   Prob(JB):                         0.00
Kurtosis:                      17.561   Cond. No.                         134.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [148]:
X = X_new[new_cols]
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
list(zip(new_cols, vif))

[('bedrooms', 21.367545650274455),
 ('bathrooms', 22.11121937658855),
 ('view', 1.2231833613284622),
 ('condition', 23.788208149284632),
 ('grade', 41.34465083936745),
 ('basement', 1.943942661711833),
 ('Renovated', 1.044358331843423),
 ('lat_log', 1.0558620311480018),
 ('long_log', 1.216866028148088),
 ('sqft_living_log', 2.020303495346973)]

In [12]:
X_all = df.drop(columns=['price', 'date', 'month', 'yr_renovated', 'yr_built', 'lat', 'long'])
rfe.fit(X_all, df['price'])

Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.


RFE(estimator=LinearRegression(copy_X=True, fit_intercept=False, n_jobs=-1,
                               normalize=False),
    n_features_to_select=10, step=1, verbose=2)

In [13]:
new_cols = X_all.columns[rfe.support_]

In [14]:
X_new = df[new_cols]
X_new.head()

,bedrooms,bathrooms,floors,view,condition,grade,basement,Renovated,year,age_when_sold
0,3,2.25,2.0,0,3,7,1,1,2014,63
1,2,1.00,1.0,0,3,6,0,0,2015,82
2,4,3.00,1.0,0,5,7,1,0,2014,49
3,3,2.00,1.0,0,3,8,0,0,2015,28
4,4,4.50,1.0,0,3,11,1,0,2014,13
